In [7]:
import pandas as pd
import jieba
from collections import Counter
import json
import re

# 停用词列表（可扩展）
stop_words = set([
    '的', '了', '是', '我', '也', '和', '就', '都', '很', '在', '有',
    '一个', '这个', '那个', '我们', '你们', '他们', '但', '被', '等',
    '上', '下', '中', '说', '着', '自己', '不会', '可以', '因为', '还',
    '让', '去', '要', '呢', '啊', '吧', '呀', '哦', '嘛', '啦','超话','人一','搜索','微博','话题','评论','点赞','转发','关注','粉丝',
    '回复', '发布', '内容', '信息', '平台', '用户', '账号', '数据', '链接', '查看','什么', '怎么', '为什么', '哪里', '谁', '多少', '如何', '怎样',
    '时间', '问题', '答案', '回答', '文章', '博文', '微博搜索', '关键词', '采集', '分析','上海','郑州','杭州','视频','其他','或者','直接','浙江','的话',
    '南京','广州','深圳','北京','天津','重庆','武汉','西安','成都','长沙','青岛','苏州','厦门','济南','合肥','无锡','所以','齐齐哈尔','淮安','身边','可能','就是','以及','一些',
    '一天','一定','一下','一般','已经','一直','一些','这种','招聘','而且','尽量','不用','不要','同好','如果','没有'
])


# 读取 Excel
df = pd.read_excel(r"C:\Users\ThinkPad\Desktop\信息可视化\期末作业\微博搜索关键词采集.xlsx")
texts = df['博文内容'].dropna().tolist()

# 清洗文本
def clean_text(text):
    text = re.sub(r'[^\u4e00-\u9fa5]', '', text)
    return text

cleaned = [clean_text(text) for text in texts]

# 分词 + 去停用词
all_words = []
for text in cleaned:
    words = jieba.cut(text)
    all_words.extend([w for w in words if len(w) > 1 and w not in stop_words])

# 统计词频
counter = Counter(all_words)
most_common = counter.most_common(100)

# 手动扩大频率差异（幂次增强）
scaled_data = []
for word, freq in most_common:
    scaled_data.append({
        "name": word,
        "value": int(freq ** 1.5)  # 增强视觉差异（可调 1.5 ~ 2.0）
    })

# 保存为 JSON
with open("weibo_word_freq.json", "w", encoding="utf-8") as f:
    json.dump(scaled_data, f, ensure_ascii=False, indent=2)

print("已过滤停用词并增强词频差异，保存至 word_freq.json")



已过滤停用词并增强词频差异，保存至 word_freq.json


In [8]:
import pandas as pd
import jieba
from collections import Counter

file_path = r'C:\Users\ThinkPad\Desktop\信息可视化\期末作业\知乎关键词搜索问答采集.xlsx'
df = pd.read_excel(file_path)

texts = df['问题标题'].astype(str).tolist() + df['回答内容'].astype(str).tolist()

# 清洗文本
def clean_text(text):
    text = re.sub(r'[^\u4e00-\u9fa5]', '', text)
    return text
cleaned_texts = [clean_text(text) for text in texts if isinstance(text, str) and text.strip()]
texts = cleaned_texts
# 分词
words = []
for text in texts:
    words.extend([w for w in jieba.cut(text) if len(w) > 1])  # 去除单字词

# 统计词频
counter = Counter(words)

# 过滤掉常见无意义词
stop_words = set([
    '的', '了', '是', '我', '也', '和', '就', '都', '很', '在', '有',
    '一个', '这个', '那个', '我们', '你们', '他们', '但', '被', '等',
    '上', '下', '中', '说', '着', '自己', '不会', '可以', '因为', '还',
    '让', '去', '要', '呢', '啊', '吧', '呀', '哦', '嘛', '啦','超话','人一','搜索','微博','话题','评论','点赞','转发','关注','粉丝',
    '回复', '发布', '内容', '信息', '平台', '用户', '账号', '数据', '链接', '查看','什么', '怎么', '为什么', '哪里', '谁', '多少', '如何', '怎样',
    '时间', '问题', '答案', '回答', '文章', '博文', '微博搜索', '关键词', '采集', '分析','上海','郑州','杭州','视频','其他','或者','直接','浙江','的话',
    '南京','广州','深圳','北京','天津','重庆','武汉','西安','成都','长沙','青岛','苏州','厦门','济南','合肥','无锡','所以','齐齐哈尔','淮安','身边','可能','就是','以及','一些',
    '一天','一定','一下','一般','已经','一直','一些','这种','招聘','而且','尽量','不用','不要','同好','如果','没有'
])

filtered = {word: freq for word, freq in counter.items() if word not in stop_words}

# 转成词云需要的格式
word_freq_list = [{"name": word, "value": freq} for word, freq in filtered.items()]

# 保存为json文件，供echarts调用
import json
with open('zhihu_word_freq.json', 'w', encoding='utf-8') as f:
    json.dump(word_freq_list, f, ensure_ascii=False)


In [9]:
import json
from collections import defaultdict

def load_word_freq(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

def merge_word_freq(list1, list2):
    merged = defaultdict(int)
    for item in list1:
        merged[item['name']] += item['value']
    for item in list2:
        merged[item['name']] += item['value']
    return [{"name": k, "value": v} for k, v in merged.items()]

weibo_data = load_word_freq('weibo_word_freq.json')
zhihu_data = load_word_freq('zhihu_word_freq.json')

merged_data = merge_word_freq(weibo_data, zhihu_data)

with open('merged_word_freq.json', 'w', encoding='utf-8') as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)
